In [3]:
import subprocess as sp
import numpy as np
import nibabel as nib
from util_func import *

from scipy.ndimage import binary_erosion

sub_sm = ['sub-01','sub-03','sub-04','sub-05','sub-08','sub-09','sub-10','sub-12','sub-14','sub-15','sub-16','sub-17',
          'sub-18','sub-20','sub-21','sub-22','sub-24','sub-25','sub-27','sub-28','sub-29','sub-30','sub-31','sub-32']

## Excluding subjects with excessive motion

In [ ]:
sub_sm_clean = []
for subj in sub_sm:
    for task in ['movie', 'recall']:
        for run in range(1,4):
            try:
                df = load_confounds(subj, task=task, run=run)
                df = extract_cols(df)
                fd = np.array(df['framewise_displacement'].values.tolist())

                framewise_displacement = fd
                percent_motion_outliers = np.sum(framewise_displacement>0.5)/fd.shape[0]
                print(subj,task,run,': ',percent_motion_outliers)
                if percent_motion_outliers > 0.05:
                    sub_sm_clean.append([subj,task,run, percent_motion_outliers])
                    print(subj,task,run,': ',percent_motion_outliers)
            except:
                pass
          

## Denosing confounds

In [ ]:
sub_sm_exclude = []
mni_mask = nib.load('./files/MNI152NLin2009cAsym_3mm_mask.nii.gz').get_fdata()
mni_image = nib.load('./files/MNI152NLin2009cAsym_3mm_mask.nii.gz')
folder_name = f'/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/SM_DATA/social_movie_recall'

for i, fname in enumerate(sub_sm_clean):
    for task in ['movie','recall']:
        for run in range(1,4):
            print('+++++++++', fname, task, run, '+++++++++')
            path = f'{folder_name}/derivatives/fmriprep/{fname}/func/{fname}_task-{task}_run-{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

            fmri_data = nib.load(path).get_fdata()
            print('original: ', fmri_data.shape)
            fmri_data = fmri_data[mni_mask==1,:].T
            print('masked: ', fmri_data.shape)

            fmri_compounds = load_confounds(fname, task, run)
            fmri_clean = clean_data(fmri_data, fmri_compounds)
            ffmri_clean = fmri_clean.astype(np.float32)

            new_image = np.zeros(mni_mask.shape+(fmri_data.shape[0],), dtype=np.float32)
            new_image[mni_mask==1,:] = fmri_clean.T

            nib.save(nib.Nifti1Image(new_image, mni_image.affine), f'{folder_name}/social_movie1/{fname}_{task}_{run}.nii.gz')


## Additional preprocessing

In [ ]:
# Additional preprocessing step

mask_fname = './files/MNI152NLin2009cAsym_3mm_mask.nii.gz'
folder_name = f'/media/dasom/caee0336-77a9-438c-af2f-7cf5b88293e7/dasom/SM_DATA/social_movie1/social_movie_recall'
FWHM = 5
for i, fname in enumerate(sub_sm):
    for task in ['movie', 'qna']:
        for run in range(1,4):
            print(fname,task,run)
            input_fname = f'{folder_name}/{fname}_{task}_{run}.nii.gz'
            mean_fname = f'{folder_name}/{fname}_{task}_{run}_mean.nii.gz'
            SC_fname = f'{folder_name}/{fname}_{task}_{run}_sc.nii.gz'
            SM_SC_fname = f'{folder_name}/{fname}_{task}_{run}_sc_sm.nii.gz'

            sp.call(f'/home/dasom/abin/3dTstat -prefix {mean_fname} {input_fname}', shell=True) # Mean calculation
            sp.call(f"/home/dasom/abin/3dcalc -a {input_fname} -b {mean_fname} -c {mask_fname} -expr 'c * min(200, a/b*100)*step(a)*step(b)' -prefix {SC_fname}", shell=True) # Scaling
            sp.call(f'/home/dasom/abin/3dmerge -quiet -1blur_fwhm {FWHM} -doall -prefix {SM_SC_fname} {SC_fname}',  shell=True) # Spatial Smoothing